In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import json
from PIL import Image
import os
import plotly.graph_objects as px
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from plotly.offline import init_notebook_mode
from plotly.offline import iplot
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.applications import DenseNet121

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


2025-08-19 03:14:08.693468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755573248.708705       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755573248.713124       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-19 03:14:08.732046: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_df = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test_df = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
subs_df = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')

image_path = '../input/siim-isic-melanoma-classification/'

print('The size of training data : {}'.format(train_df.shape))
print('The size of testing data : {}'.format(test_df.shape))


The size of training data : (28984, 8)
The size of testing data : (4142, 5)


In [4]:
train_df.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
3,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
4,ISIC_0074311,IP_2950485,female,40.0,lower extremity,unknown,benign,0


In [5]:
plt.figure(figsize = (17,7))
percent_missing = train_df.isnull().sum() / (train_df.shape[0])*100
percent_missing.iplot(kind = 'bar',color ='blue')


<Figure size 1700x700 with 0 Axes>

In [6]:
train_df.describe()


,age_approx,target
count,28925.000000,28984.000000
mean,48.886776,0.017699
std,14.408609,0.131859
min,0.000000,0.000000
25%,40.000000,0.000000
50%,50.000000,0.000000
75%,60.000000,0.000000
max,90.000000,1.000000


In [7]:
print("The total patient ids are {}, from those the unique ids are {}".format(train_df['patient_id'].count(),train_df['patient_id'].value_counts().shape[0] ))


The total patient ids are 28984, from those the unique ids are 2056


In [8]:
benign_gender = train_df.groupby(['benign_malignant']).count()['sex'].to_frame()
benign_gender.head()


,sex
benign_malignant,
benign,28415
malignant,513


In [9]:
feature_list = ['sex','age_approx','anatom_site_general_challenge','benign_malignant'] 
for i in feature_list: 
    train_df[i].value_counts(normalize=True).to_frame().iplot(kind='bar',
                                                      yTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='blue',
                                                      theme='pearl',
                                                      bargap=0.8,
                                                      gridcolor='white',                                                     
                                                      title=f'Distribution of {i} in train set.')

    test_df[i].value_counts(normalize=True).to_frame().iplot(kind='bar',
                                                      yTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='green',
                                                      theme='pearl',
                                                      bargap=0.8,
                                                      gridcolor='white',                                                     
                                                      title=f'Distribution of {i} in test set.')


KeyError: 'benign_malignant'

In [10]:
subs_df.to_csv('submissions.csv', index = False)
print('Successfull!!')


Successfull!!
